In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from colorama import Fore, Style, init
from scipy.optimize import minimize
import warnings
import copy

In [ ]:
init(autoreset=True)
warnings.filterwarnings('ignore')

import torch

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print("Using device:", device)

if device.type == 'cuda':
    xgb_device = 'cuda'
else:
    xgb_device = 'cpu'

print("XGBoost device:", xgb_device)

SEED = 42
FOLDS = 7
ALPHA = 0.1
MIN_COVERAGE = 0.90

print("Enhanced Two-Stage Uncertainty Model")
print("Minimum Coverage:", MIN_COVERAGE)

In [ ]:
def winkler_score(y_true, lower, upper, alpha=0.1, return_coverage=False):

    y_true = np.asarray(y_true)
    lower = np.asarray(lower)
    upper = np.asarray(upper)

    width = upper - lower

    penalty_lower = 2 / alpha * (lower - y_true)
    penalty_upper = 2 / alpha * (y_true - upper)

    score = width \
        + np.where(y_true < lower, penalty_lower, 0) \
        + np.where(y_true > upper, penalty_upper, 0)

    if return_coverage:
        coverage = np.mean((y_true >= lower) & (y_true <= upper))
        return np.mean(score), coverage

    return np.mean(score)


def robust_winkler_objective(gammas, y_true, y_hat, err_hat,
                             min_coverage=0.90,
                             coverage_penalty=1000000):

    gamma0 = gammas[0]
    gamma1 = gammas[1]

    err_term = np.sqrt(err_hat)

    lower = y_hat - gamma0 * err_term
    upper = y_hat + gamma1 * err_term

    score, coverage = winkler_score(
        y_true, lower, upper, return_coverage=True
    )

    if coverage < min_coverage:
        penalty = coverage_penalty * (min_coverage - coverage) ** 2
        score = score + penalty

    return score


In [ ]:
def preprocess_date(data):

    df = data.copy()

    df["sale_date"] = pd.to_datetime(df["sale_date"], errors='coerce')

    df["year"] = df["sale_date"].dt.year
    df["month"] = df["sale_date"].dt.month
    df["quarter"] = df["sale_date"].dt.quarter
    df["day"] = df["sale_date"].dt.day
    df["dayofweek"] = df["sale_date"].dt.dayofweek

    df["is_weekend"] = df["dayofweek"].isin([5, 6]).astype(int)
    df["week_of_year"] = df["sale_date"].dt.isocalendar().week

    df["month_sin"] = np.sin(2 * np.pi * df["month"] / 12)
    df["month_cos"] = np.cos(2 * np.pi * df["month"] / 12)

    df["day_sin"] = np.sin(2 * np.pi * df["day"] / 31)
    df["day_cos"] = np.cos(2 * np.pi * df["day"] / 31)

    df["week_sin"] = np.sin(2 * np.pi * df["week_of_year"] / 52)
    df["week_cos"] = np.cos(2 * np.pi * df["week_of_year"] / 52)

    df["is_start_of_month"] = (df["day"] <= 7).astype(int)
    df["is_end_of_month"] = (df["day"] >= 24).astype(int)

    df["imp_to_land_ratio"] = df["imp_val"] / (df["land_val"] + 1)
    df["total_val"] = df["imp_val"] + df["land_val"]

    df["val_per_sqft"] = df["total_val"] / (df["sqft"].replace(0, np.nan) + 1)

    df["lot_to_building_ratio"] = df["sqft_lot"] / (df["sqft"] + 1)

    df["log_total_val"] = np.log1p(df["total_val"])
    df["log_sqft"] = np.log1p(df["sqft"])

    if "year_built" in df.columns:
        df["property_age"] = df["year"] - df["year_built"]

    df.drop(["sale_date"], axis=1, inplace=True)

    return df


In [ ]:
def retrieve_neighbours(model, X, y, k, exclude_self):

    if exclude_self:
        distances, indices = model.kneighbors(X, n_neighbors=k+1)
    else:
        distances, indices = model.kneighbors(X, n_neighbors=k)

    preds = []
    dists = []
    stds = []

    for d, idxs in zip(distances, indices):

        if exclude_self:
            d = d[1:]
            idxs = idxs[1:]

        vals = y[idxs]

        preds.append(np.mean(vals))
        dists.append(np.mean(d))

        if len(vals) > 1:
            stds.append(np.std(vals))
        else:
            stds.append(0)

    return np.array(preds), np.array(dists), np.array(stds)


def preprocess_knn_features(X_tr, X_va, y_tr, knn_features, knn_params):

    scaler = StandardScaler()

    X_tr_knn = scaler.fit_transform(X_tr[knn_features])
    X_va_knn = scaler.transform(X_va[knn_features])

    X_tr_out = X_tr.copy()
    X_va_out = X_va.copy()

    for k in [5, 10, 15]:

        params = knn_params.copy()
        params["n_neighbors"] = k

        knn = KNeighborsRegressor(**params)
        knn.fit(X_tr_knn, y_tr)

        p_tr, d_tr, s_tr = retrieve_neighbours(knn, X_tr_knn, y_tr, k, True)
        p_va, d_va, s_va = retrieve_neighbours(knn, X_va_knn, y_tr, k, False)

        X_tr_out["price_knn_" + str(k)] = p_tr
        X_tr_out["dist_knn_" + str(k)] = d_tr
        X_tr_out["std_knn_" + str(k)] = s_tr

        X_va_out["price_knn_" + str(k)] = p_va
        X_va_out["dist_knn_" + str(k)] = d_va
        X_va_out["std_knn_" + str(k)] = s_va

    return X_tr_out, X_va_out


In [ ]:
def fit_two_stage_model(X, y, model0, model1, n_splits, seed):

    y = np.asarray(y)
    oof = np.zeros(len(y))

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)

    for tr_idx, va_idx in kf.split(X):

        X_tr = X.iloc[tr_idx]
        X_va = X.iloc[va_idx]
        y_tr = y[tr_idx]
        y_va = y[va_idx]

        model0.fit(X_tr, y_tr)
        oof[va_idx] = model0.predict(X_va)

    resid_target = (y - oof) ** 2 + 1e-6

    X_resid = X.copy()
    X_resid["y_pred"] = oof
    X_resid["y_pred_sq"] = oof ** 2

    model1.fit(X_resid, resid_target)

    model0.fit(X, y)

    return model0, model1


In [ ]:
def predict_components(X, model0, model1, lower_bound):

    y_hat = model0.predict(X)

    X_resid = X.copy()
    X_resid["y_pred"] = y_hat
    X_resid["y_pred_sq"] = y_hat ** 2

    err_hat = model1.predict(X_resid)

    err_hat = np.maximum(err_hat, lower_bound)

    return y_hat, err_hat


def build_intervals(y_hat, err_hat, g0, g1):

    err_term = np.sqrt(err_hat)

    lower = y_hat - g0 * err_term
    upper = y_hat + g1 * err_term

    return lower, upper


In [ ]:
train = pd.read_csv("dataset.csv")
test = pd.read_csv("test.csv")

train = preprocess_date(train)
test = preprocess_date(test)

cat_cols = []
for c in train.columns:
    if train[c].dtype == "object" and c != "sale_price":
        cat_cols.append(c)

encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)

train[cat_cols] = encoder.fit_transform(train[cat_cols]).astype(int)
test[cat_cols] = encoder.transform(test[cat_cols]).astype(int)

print("Train shape:", train.shape)
print("Test shape:", test.shape)


In [ ]:
knn_params = {"weights": "distance"}

y = train["sale_price"]

scores = []
coverages = []

kf = KFold(n_splits=FOLDS, shuffle=True, random_state=SEED)

for fold, (tr_idx, va_idx) in enumerate(kf.split(train), 1):

    print("Fold", fold)

    X_tr = train.iloc[tr_idx]
    X_va = train.iloc[va_idx]

    y_tr = y.iloc[tr_idx]
    y_va = y.iloc[va_idx]

    X_tr, X_va = preprocess_knn_features(
        X_tr, X_va, y_tr,
        ["latitude", "longitude", "year"],
        knn_params
    )

    model0 = XGBRegressor(device=xgb_device)
    model1 = XGBRegressor(objective="reg:gamma", device=xgb_device)

    model0, model1 = fit_two_stage_model(
        X_tr, y_tr, model0, model1, FOLDS, SEED
    )

    y_hat, err_hat = predict_components(X_va, model0, model1, 1000)

    lower, upper = build_intervals(y_hat, err_hat, 1.5, 1.6)

    score, cov = winkler_score(y_va, lower, upper, return_coverage=True)

    print("Winkler:", round(score), "Coverage:", round(cov, 4))

    scores.append(score)
    coverages.append(cov)


In [ ]:
print("Optimizing gammas")

best_score = 1e18
best_g0 = 1.5
best_g1 = 1.6

for i in range(3):

    g0 = np.random.uniform(1.2, 2.0)
    g1 = np.random.uniform(1.2, 2.0)

    score = robust_winkler_objective(
        [g0, g1], y_va, y_hat, err_hat, MIN_COVERAGE
    )

    print("Try", i+1, "g0", g0, "g1", g1, "score", round(score))

    if score < best_score:
        best_score = score
        best_g0 = g0
        best_g1 = g1

print("Best gammas:", best_g0, best_g1)


In [ ]:
X_train, X_test = preprocess_knn_features(
    train, test, y,
    ["latitude", "longitude", "year"],
    knn_params
)

model0 = XGBRegressor(device=xgb_device)
model1 = XGBRegressor(objective="reg:gamma", device=xgb_device)

model0, model1 = fit_two_stage_model(
    X_train, y, model0, model1, FOLDS, SEED
)

print("Predicting")

_, lower, upper = build_intervals(
    *predict_components(X_test, model0, model1, 1000),
    best_g0,
    best_g1
)

submission = pd.DataFrame({
    "id": test.index,
    "pi_lower": lower,
    "pi_upper": upper
})

submission.to_csv("submission.csv", index=False)
print("Submission saved!")
